<a href="https://colab.research.google.com/github/newtonfalbo/dio-lab-open-source/blob/main/transfer_learning_Newton%20Borges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import tensorflow as tf
    import tensorflow_datasets as tfds
    import matplotlib.pyplot as plt
    import numpy as np
    from tensorflow.keras import layers, models
except ModuleNotFoundError as e:
    print("Erro ao importar módulos. Certifique-se de que o TensorFlow está instalado.")
    raise e

# Configuração do ambiente
print("Versão do TensorFlow:", tf.__version__)

def carregar_dataset():
    # Carregar dataset de gatos e cachorros
    (ds_train, ds_test), ds_info = tfds.load(
        'cats_vs_dogs',
        split=['train[:80%]', 'train[80%:]'],
        as_supervised=True,
        with_info=True
    )
    return ds_train, ds_test, ds_info

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0  # Normalização
    return image, label

def preparar_datasets(ds_train, ds_test):
    ds_train = ds_train.map(preprocess_image).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)
    ds_test = ds_test.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)
    return ds_train, ds_test

def construir_modelo():
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Congela o modelo base

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Executar pipeline
ds_train, ds_test, ds_info = carregar_dataset()
ds_train, ds_test = preparar_datasets(ds_train, ds_test)
modelo = construir_modelo()

# Treinamento
history = modelo.fit(ds_train, validation_data=ds_test, epochs=5)

# Avaliação
loss, acc = modelo.evaluate(ds_test)
print(f'Acurácia do modelo: {acc:.2f}')

# Salvar modelo
modelo.save("transfer_learning_model.h5")

Versão do TensorFlow: 2.18.0
Epoch 1/5
410/582 ━━━━━━━━━━━━━━━━━━━━ 3:48 1s/step - accuracy: 0.9304 - loss: 0.1758